![tittle](img/universidad_politecnica_salesiana.png)
# <center>Systems Engineering Career</center><br><center>Artificial Intelligence II</center><br><center>Neural Networks with Scikit-Learn: Shape Recognition</center>

**By: Jorge Sanisaca**

## Introduction

This book presents a project on the main aspects to create, train and validate articular neural networks in Ptython with Scikit-Learn. Taking into account multilayer Perceptron as a tool to perform the classification.

## Prerequisites

We will need to have the following libraries installed:

1. Python (versiones >=2.7 o >=3.3)
2. Numpy >= 1.16.5
3. SciPy >= 0.13.3
4. Scikit-Learn 0.21.3
5. Pandas >= 0.25.1
6. viznet

## Installations

    pip install -U scikit-learn
    pip install viznet
    pip install pandas

## Problema

The corpus represents a set of data a set of data of the weighted means of the pixel intensities of the images already processed with <a href='https://opencv.org/'>OpenCV</a>, where each column represents the moments that are invariable for the translation, the scale and the rotation that are called **Hu Moments** which are a set of 7 numbers calculated using central moments that are invariable for image transformations.

The objective is to design and train a neural network that allows classifying samples based on the characteristics of these images.

## Process

### Data reading

As a first step we will proceed to load the data using the <a href='https://pandas.pydata.org/'>Pandas</a> library. To do this, we will use the read_csv method and specify the separator and the names we want to be loaded when reading the file.

It is important to note that the read_csv method returns a dataframe object.


In [1]:
import pandas as pd
import numpy as np

%matplotlib inline
datos = pd.read_csv('corpus/datos.csv', sep=";")
datos.head()

,f1,f2,f3,f4,f5,f6,f7,Fichero
0,0.161964,0.000004,4.605700e-05,2.705150e-06,3.012040e-11,5.001100e-09,-2.122570e-12,train/apple-1.png
1,0.236366,0.000105,1.099700e-02,7.623680e-05,-3.180690e-08,6.968630e-07,-6.213690e-08,train/device8-3.png
2,0.159180,0.000008,2.945610e-10,2.174170e-14,5.484940e-26,5.958650e-17,4.340170e-27,train/device9-20.png
3,0.187058,0.003204,5.355570e-05,1.366400e-05,-6.237880e-11,4.878330e-07,3.643300e-10,train/Misk-9.png
4,0.176794,0.000002,6.260300e-07,1.835560e-09,-1.577080e-17,8.472220e-13,-6.019110e-17,train/device1-10.png


### Get the name of the image

As we can see the dataframe _data_ generated in the previous step we can see that the column _File_ contains the path where the image is located, however for this processing we need to stay only with the name or type of image for this we will cut the column in order to stay alone with the name of the image.

In [2]:
datos['Fichero'] = datos['Fichero'].str.split('/').str[1]
datos['Fichero'] = datos['Fichero'].str.split('-').str[0]

datos.head()

,f1,f2,f3,f4,f5,f6,f7,Fichero
0,0.161964,0.000004,4.605700e-05,2.705150e-06,3.012040e-11,5.001100e-09,-2.122570e-12,apple
1,0.236366,0.000105,1.099700e-02,7.623680e-05,-3.180690e-08,6.968630e-07,-6.213690e-08,device8
2,0.159180,0.000008,2.945610e-10,2.174170e-14,5.484940e-26,5.958650e-17,4.340170e-27,device9
3,0.187058,0.003204,5.355570e-05,1.366400e-05,-6.237880e-11,4.878330e-07,3.643300e-10,Misk
4,0.176794,0.000002,6.260300e-07,1.835560e-09,-1.577080e-17,8.472220e-13,-6.019110e-17,device1


We can also analyze the corpus using the function described by Pandas. The values that we can obtain from each variable are the following:

* count
* mean
* standard deviation (std)
* min
* Percentiles, which are the values that are between 25%, 50%, 75%.
* max

In [3]:
datos.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
f1,1120.0,0.317706,0.200907,0.159180,1.991075e-01,2.493580e-01,3.419902e-01,1.550570
f2,1120.0,0.083006,0.224974,0.000000,2.028440e-03,1.621600e-02,5.150630e-02,2.162990
f3,1120.0,0.013257,0.047733,0.000000,2.764270e-04,1.070585e-03,5.188960e-03,0.942444
f4,1120.0,0.005712,0.023760,0.000000,2.288693e-05,1.352400e-04,8.837387e-04,0.327414
f5,1120.0,0.000791,0.006923,-0.000156,-5.286663e-14,4.632130e-09,3.788562e-07,0.181674
f6,1120.0,0.003449,0.017738,-0.004850,-7.138183e-09,2.447115e-06,8.138190e-05,0.203791
f7,1120.0,0.000003,0.000307,-0.003646,-9.976037e-09,2.349400e-13,1.558542e-08,0.008558


As we can see, the last column contains text strings that describe the type of image. To do this, we can use the map function provided by Pandas and replace the strings with numerical values that can be understood by the neural network.

In [4]:
nombres = datos['Fichero'].unique().tolist()
# print(nombres)
datos['Fichero']=datos['Fichero'].map({'apple':0, 'device8':1, 'device9':2, 'Misk':3, 'device1':4, 'carriage':5, 
                                       'device3':6, 'fly':7, 'cup':8, 'car':9, 'camel':10, 'Bone':11, 'turtle':12, 
                                       'bat':13, 'cellular_phone':14, 'pocket':15, 'device4':16, 'teddy':17, 'frog':18, 
                                       'lizzard':19, 'cattle':20, 'spoon':21, 'guitar':22, 'fountain':23, 'octopus':24, 
                                       'bird':25, 'ray':26, 'spring':27, 'chopper':28, 'horse':29, 'dog':30, 'hat':31, 
                                       'personal_car':32, 'butterfly':33, 'device5':34, 'brick':35, 'device0':36, 'key':37, 
                                       'crown':38, 'fish':39, 'shoe':40, 'Comma':41, 'fork':42, 'HCircle':43, 'device7':44, 
                                       'lmfish':45, 'watch':46, 'beetle':47, 'bell':48, 'rat':49, 'elephant':50, 'deer':51, 
                                       'hammer':52, 'jar':53, 'Heart':54, 'flatfish':55, 'sea_snake':56, 'horseshoe':57, 
                                       'Glas':58, 'tree':59, 'stef':60, 'truck':61, 'children':62, 'face':63, 'bottle':64, 
                                       'pencil':65, 'chicken':66, 'device2':67, 'device6':68, 'classic':69})
datos.head()

,f1,f2,f3,f4,f5,f6,f7,Fichero
0,0.161964,0.000004,4.605700e-05,2.705150e-06,3.012040e-11,5.001100e-09,-2.122570e-12,0
1,0.236366,0.000105,1.099700e-02,7.623680e-05,-3.180690e-08,6.968630e-07,-6.213690e-08,1
2,0.159180,0.000008,2.945610e-10,2.174170e-14,5.484940e-26,5.958650e-17,4.340170e-27,2
3,0.187058,0.003204,5.355570e-05,1.366400e-05,-6.237880e-11,4.878330e-07,3.643300e-10,3
4,0.176794,0.000002,6.260300e-07,1.835560e-09,-1.577080e-17,8.472220e-13,-6.019110e-17,4


### Neural Network Design

As a next point, we will design a neural network (multilayer perceptron classifier) to learn how to distinguish between different types of images.

The artificial neural network will have the following characteristics:

* Entradas: 7
* Número de capas: 70
* Neuronas en la capa oculta: 10
* Neuronas en la capa de salida: 70

### Preprocessing of data and generation of training and testing corpus

As a previous step to train the neural network, it is essential to preprocess the data (scale, change formats, etc.), since otherwise optimal results will not be obtained in the classification process.

In [5]:
from viznet import connecta2a, node_sequence, NodeBrush, EdgeBrush, DynamicShow
# We import the function to scale the values
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import DictVectorizer as DV
# We import the function to separate test and train
from sklearn.model_selection import train_test_split

# We create variables with the parameters that the network will have
entradas = 7
neuronas_capa_oculta = 100
neuronas_capa_salida = 70

# We separate the input data into a variable, for this we generate a copy of the dataframe
# removing the last column of the corpus (the one with the types of images)
X=datos.drop('Fichero',axis=1)

# We proceed in the same way, but in this case to generate an arrangement that has the desired outputs
d=datos['Fichero']

# We show the first data on the screen with the 'head' function
X.head()
d.head()

X_train, X_test, d_train, d_test = train_test_split(X,d,train_size=0.80,random_state=0,stratify=d)

# We perform OneHotEncoding to have 70 outputs instead of 1 and then perform a dataframe with pandas
dat_dict=datos.T.to_dict().values()
vectorizer = DV(sparse = False)
vectorizer.fit(dat_dict)
dat= vectorizer.transform(dat_dict)
dat=pd.DataFrame(dat)

# We generate an object to scale the values
scaler=StandardScaler()

# Adjust only in training data
scaler.fit(X_train)

# Escalamos el corpus de entrenamiento
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)

# We visualize the first 7 rows of data
X_train[1:7,:]

array([[-0.77011187, -0.37188139, -0.26469383, -0.23496323, -0.11352916,
        -0.19431574, -0.00430886],
       [ 3.671591  ,  3.7595119 ,  1.32166184,  0.12063621, -0.0890712 ,
         0.16902901, -0.42689269],
       [-0.67212809, -0.36312184, -0.25866301, -0.23438666, -0.11352907,
        -0.19434883, -0.00431089],
       [-0.39786821, -0.24949913, -0.26070641, -0.23268111, -0.11352843,
        -0.19409487, -0.00431634],
       [ 0.21865912, -0.36944091, -0.25543829, -0.23014452, -0.1135313 ,
        -0.19421029, -0.00423988],
       [ 0.42812264,  0.10609515, -0.15276291, -0.07138302, -0.11099275,
        -0.12594532, -0.00854249]])

In [6]:
# We import the Perceptron Multilayer for Classification
from sklearn.neural_network import MLPClassifier

# We create the neural network
mlp=MLPClassifier(solver = 'lbfgs', activation='tanh', verbose=True, alpha=1e-4, tol=1e-15, max_iter=10000, \
                  hidden_layer_sizes=(neuronas_capa_oculta, neuronas_capa_salida))

print(mlp)
# We carry out the training process
mlp.fit(X_train,d_train)

MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100, 70), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=10000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='lbfgs',
              tol=1e-15, validation_fraction=0.1, verbose=True,
              warm_start=False)


C:\Users\sanix\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100, 70), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=10000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='lbfgs',
              tol=1e-15, validation_fraction=0.1, verbose=True,
              warm_start=False)

### Prediction and evaluation of the network

The last step is to evaluate the operation of the network. To do this, we will determine how it behaves in prediction tasks with the test part (X_test, d_test):

In [7]:
from sklearn.metrics import classification_report, confusion_matrix

#print(d_test.value_counts())

prediccion = mlp.predict(X_test)
print('Matriz de Confusion\n')
print(confusion_matrix(d_test, prediccion))
print('\n')

print(classification_report(d_test, prediccion))

Matriz de Confusion

[[2 0 0 ... 0 0 0]
 [0 4 0 ... 0 0 0]
 [0 0 1 ... 1 1 0]
 ...
 [0 0 1 ... 2 0 0]
 [0 0 2 ... 0 0 0]
 [0 0 0 ... 0 0 3]]


              precision    recall  f1-score   support

           0       1.00      0.67      0.80         3
           1       1.00      1.00      1.00         4
           2       0.20      0.33      0.25         3
           3       0.33      0.67      0.44         3
           4       1.00      1.00      1.00         3
           5       0.75      1.00      0.86         3
           6       1.00      0.50      0.67         4
           7       0.50      0.33      0.40         3
           8       1.00      0.67      0.80         3
           9       0.67      0.50      0.57         4
          10       0.00      0.00      0.00         3
          11       1.00      0.33      0.50         3
          12       0.50      0.25      0.33         4
          13       0.60      1.00      0.75         3
          14       0.33      0.33      0.33   

C:\Users\sanix\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
